## Expansions and Todos
Refine no-trump logic

In [19]:
import random
random.seed(8266)
from enum import Enum
from collections.abc import Callable
import math

import dds
import ctypes
import functions
import cards
from structs import Card, Suit, Bid, BidSuit

In [20]:
POWER_TO_DENSITY = {
        1: 4,
        2: 4,
        3: 5,
        4: 6,
        5: 7,
        6: 7,
        7: 8
    }

declaredBids = [Bid(BidSuit.DIAMONDS, 1), Bid(BidSuit.PASS, 0), Bid(BidSuit.HEARTS, 1), Bid(BidSuit.PASS, 0), Bid(BidSuit.NOTRUMP, 1), Bid(BidSuit.PASS, 1), Bid(BidSuit.HEARTS, 2), Bid(BidSuit.PASS, 1), Bid(BidSuit.PASS, 1), Bid(BidSuit.PASS, 1)]
hands = [[] for _ in range(4)]

knownCards = [
    [], 
    [], 
    [Card(Suit.SPADES, 14), Card(Suit.SPADES, 13), Card(Suit.SPADES, 9), Card(Suit.HEARTS, 10), Card(Suit.HEARTS, 9), Card(Suit.HEARTS, 8), Card(Suit.HEARTS, 5), Card(Suit.HEARTS, 4), Card(Suit.CLUBS, 13), Card(Suit.CLUBS, 6), Card(Suit.DIAMONDS, 11), Card(Suit.DIAMONDS, 9), Card(Suit.DIAMONDS, 8)], 
    []
]
playedCards = [[], [], [], []]

In [21]:
def getBidComposition(bidArray: list[Bid]) -> dict[BidSuit, list[Bid]]:
    compositionDict: dict[BidSuit, list[Bid]] = dict()
    for suit in BidSuit:
        compositionDict[suit] = []
    for bid in bidArray:
        if bid.suit == BidSuit.PASS: continue
        compositionDict[bid.suit].append(bid)
    return compositionDict

In [22]:
def draw(approvalFunction: Callable[[Card], bool], draws: int):
    approvedCards = [x for x in filter(approvalFunction, deck)]
    chosenCards = random.sample(approvedCards, max(0, min(len(deck), draws)))
    for card in chosenCards:
        deck.remove(card)
    return chosenCards

In [23]:
def add_to_hand(cardList: list[Card], enterIndex: int):
    hands[enterIndex].extend(cardList)
    for card in cardList:
        suits_in_hands[enterIndex][card.suit] += 1

In [24]:
scoreDensityTable = {}

for i in range(1000):
    deck = [Card(suit, rank) for rank in range(2, 15) for suit in Suit]

    suits_in_hands = [{suit: 0 for suit in Suit} for _ in range(4)]

    # We assume that the leading hand is north. If this is not true, the board can be easily rearranged to satisfy such a condition.
    for i, known_cards_by_hand in enumerate(knownCards):
        for card in known_cards_by_hand:
            deck.remove(card)
        add_to_hand(known_cards_by_hand, i)

    bidsByHand = []
    for playerPlacement in range(4):
        bidsByHand.append([bid for bidIndex, bid in enumerate(declaredBids) if bidIndex % 4 == playerPlacement])

    finalBidIndex = -1
    for index in range(len(declaredBids) - 1, -1, -1):
        if declaredBids[index].suit != BidSuit.PASS:
            finalBidIndex = index
            break
    finalBid = declaredBids[finalBidIndex]

    # Loop through suits and determine the last played bid of that suit
    for target_suit in Suit:
        finalBidIndex = -1
        for index in range(len(declaredBids) - 1, -1, -1):
            if declaredBids[index].suit.name == target_suit.name:
                finalBidIndex = index
                break
        # Suits don't NEED to be bid so they can be skipped if never bid
        if finalBidIndex < 0: continue

        # For statekeeping, I've labeled the final bidder of a suit the "major" and the partner of that bidder the "minor".
        major_player_index = finalBidIndex % 4
        minor_player_index = (major_player_index + 2) % 4
        bidCompositions = [x for x in map(getBidComposition, bidsByHand)]

        major_bid_composiiton, minor_bid_composition = bidCompositions[major_player_index], bidCompositions[minor_player_index]
        major_bid = declaredBids[finalBidIndex]

        major_suit_bids, minor_suit_bids = major_bid_composiiton[major_bid.suit], minor_bid_composition[major_bid.suit]
        totalSum = 0
        major_quantity = 0
        minor_quantity = 0
        if len(major_suit_bids) > 0:
            major_quantity = POWER_TO_DENSITY[major_suit_bids[-1].rank] + len(major_suit_bids) - 1
        if len(minor_suit_bids) > 0:
            minor_quantity = (POWER_TO_DENSITY[minor_suit_bids[-1].rank] + len(minor_suit_bids) - 1) * 1/2
        totalSum = major_quantity + minor_quantity
        if major_quantity != 0:
            normalizationConstant = major_quantity / totalSum
            major_normalized = math.floor(major_quantity * normalizationConstant)

            major_expected = major_normalized
            minor_expected = major_quantity - major_expected

            major_add = min(major_expected - suits_in_hands[major_player_index][target_suit], 13 - len(hands[major_player_index]))
            minor_add = min(minor_expected - suits_in_hands[minor_player_index][target_suit], 13 - len(hands[minor_player_index]))

            add_to_hand(draw(lambda x: x.suit == target_suit, major_add), major_player_index)
            add_to_hand(draw(lambda x: x.suit == target_suit, minor_add), minor_player_index)

    for i, hand in enumerate(hands):
        add_to_hand(draw(lambda x: True, 13 - len(hand)), i)

    dl = dds.deal()
    fut2 = dds.futureTricks()
    fut3 = dds.futureTricks()

    threadIndex = 0
    line = ctypes.create_string_buffer(80)

    dds.SetMaxThreads(ctypes.c_longlong(0))

    dl.trump = finalBid.suit.value
    dl.first = 0

    dl.currentTrickSuit[0] = 0
    dl.currentTrickSuit[1] = 0
    dl.currentTrickSuit[2] = 0

    dl.currentTrickRank[0] = 0
    dl.currentTrickRank[1] = 0
    dl.currentTrickRank[2] = 0

    # most needed constant variables:
    for player_index in range(dds.DDS_HANDS):
        remaining_cards = hands[player_index].copy()
        for card in playedCards[player_index]:
            remaining_cards.remove(card)
        for suit_index in range(dds.DDS_SUITS):
            subset_cards = [x for x in filter(lambda x: x.suit.value == suit_index, remaining_cards)]
            dl.remainCards[player_index][suit_index] = cards.to_holding_notation(subset_cards)

        ###
        # target: Either a number <= 13 or 0 or -1: The 'target' number of tricks to make
        # solutions: 1 - 3: 1 means return 1 solution, 
        #   2 means return all maximum solutions, 3 means all solutions
        # mode: 0 or 1. Just for internal use. 0 is fine here.
        ###
    target = -1 # No target; find all results
    mode = 0 # The way dds internally handles the last trick

    solutions = 1 # Return only the optmial solutions
    res = dds.SolveBoard(dl, target, solutions, mode, ctypes.pointer(fut2), threadIndex)
    if res != dds.RETURN_NO_FAULT:
        dds.ErrorMessage(res, "Failed to evaluate:")
        print("DDS error: {}".format(line.value.decode("utf-8")))

    expected_score = ctypes.pointer(fut2).contents.score[0]
    if expected_score not in scoreDensityTable: scoreDensityTable[expected_score] = 0
    scoreDensityTable[expected_score] += 1
print(scoreDensityTable)


{7: 9, 6: 989, 8: 2}
